## Thuật toán sử dụng phương pháp newton tiến cho phương pháp giải đa thức nội suy 

In [18]:
import numpy as np
import pandas as pd
import math
from numpy.polynomial import polynomial as P

def load_horizontal_data(filepath):
    """
    Hàm đọc dữ liệu dạng hàng ngang từ file txt một cách mạnh mẽ.
    """
    try:
        with open(filepath, 'r', encoding='utf-8-sig') as f:
            line = f.readline()
        if not line: return np.array([])
        line = line.strip().replace(',', '.')
        data = np.fromstring(line, sep=' ')
        return data
    except FileNotFoundError:
        raise IOError(f"LỖI: Không tìm thấy file '{filepath}'.")
    except Exception as e:
        raise IOError(f"LỖI: Không thể đọc file '{filepath}'. Lỗi: {e}")

# --- Chương trình chính cho Ô nạp dữ liệu ---
try:
    # 1. Nạp dữ liệu thô
    x_coords_raw = load_horizontal_data('FileX.txt')
    y_coords_raw = load_horizontal_data('FileY.txt')

    if len(x_coords_raw) == 0 or len(y_coords_raw) == 0:
        raise ValueError("LỖI: Một trong hai file không chứa dữ liệu số hợp lệ.")
    if len(x_coords_raw) != len(y_coords_raw):
        raise ValueError("LỖI: Số lượng điểm x và y không khớp.")
    if len(np.unique(x_coords_raw)) != len(x_coords_raw):
        raise ValueError("LỖI: Các giá trị x bị trùng lặp.")

    # 2. SẮP XẾP DỮ LIỆU TĂNG DẦN (cho Newton Tiến)
    print("Đang sắp xếp dữ liệu theo x tăng dần...")
    sort_indices_ascending = np.argsort(x_coords_raw)
    x_coords = x_coords_raw[sort_indices_ascending]
    y_coords = y_coords_raw[sort_indices_ascending]

    # 3. KIỂM TRA MỐC CÁCH ĐỀU (THEO YÊU CẦU)
    diffs = np.diff(x_coords) # Tính khoảng cách giữa các điểm
    h_step = diffs[0] # Lấy bước nhảy đầu tiên
    # Kiểm tra xem tất cả các bước nhảy có "gần bằng" bước nhảy đầu tiên không
    if not np.allclose(diffs, h_step, rtol=1e-08, atol=1e-10):
        raise ValueError(f"LỖI: Các mốc nội suy x không cách đều nhau. (Ví dụ: {diffs})")

    # 4. In kết quả đã sắp xếp
    print("Dữ liệu đã được đọc, sắp xếp và kiểm tra cách đều thành công!")
    print(f"Tổng số {len(x_coords)} điểm đã được nạp. Bước nhảy h = {h_step:g}")
    print("-" * 50)
    print("Các mốc x (đã sắp xếp tăng dần):", x_coords.tolist())
    print("Các giá trị y (tương ứng):", y_coords.tolist())
    
except (IOError, ValueError) as e:
    print(str(e))

Đang sắp xếp dữ liệu theo x tăng dần...
Dữ liệu đã được đọc, sắp xếp và kiểm tra cách đều thành công!
Tổng số 6 điểm đã được nạp. Bước nhảy h = 0.2
--------------------------------------------------
Các mốc x (đã sắp xếp tăng dần): [2.1, 2.3, 2.5, 2.7, 2.9, 3.1]
Các giá trị y (tương ứng): [0.5597, 1.0378, 1.5337, 1.9847, 2.3355, 2.5465]


In [19]:
def create_forward_difference_table(y_values):
    """
    Tạo bảng sai phân tiến (dạng ma trận tam giác trên).
    Các hệ số Newton (Δ^k y_0) nằm ở hàng đầu tiên: table[0, :]
    """
    n = len(y_values)
    table = np.zeros([n, n])
    table[:, 0] = y_values # Cột đầu tiên là y
    
    for j in range(1, n): # Cột
        for i in range(n - j): # Hàng
            table[i, j] = table[i+1, j-1] - table[i, j-1]
            
    return table

# Tạo bảng từ dữ liệu ĐÃ SẮP XẾP
diff_table_for_calc = create_forward_difference_table(y_coords)

# Lấy các hệ số Newton Tiến (HÀNG ĐẦU TIÊN của bảng)
coeffs_delta_y0 = diff_table_for_calc[0, :]

# Hiển thị bảng
df_diff = pd.DataFrame(diff_table_for_calc)
print("--- Bảng Sai phân Tiến (Δ^k y_i) ---")
display(df_diff.style.format("{:g}", na_rep=""))

print("\n" + "="*50)
print("--- Các hệ số Sai phân Tiến (Hàng đầu tiên của bảng, Δ^k y_0) ---")
print(coeffs_delta_y0)

--- Bảng Sai phân Tiến (Δ^k y_i) ---


,0,1,2,3,4,5
0,0.5597,0.4781,0.0178,-0.0627,0.0074,0.0083
1,1.0378,0.4959,-0.0449,-0.0553,0.0157,0
2,1.5337,0.451,-0.1002,-0.0396,0,0
3,1.9847,0.3508,-0.1398,0,0,0
4,2.3355,0.211,0,0,0,0
5,2.5465,0,0,0,0,0



--- Các hệ số Sai phân Tiến (Hàng đầu tiên của bảng, Δ^k y_0) ---
[ 0.5597  0.4781  0.0178 -0.0627  0.0074  0.0083]


In [20]:
# Giả định các biến x_coords và y_coords đã tồn tại từ Ô 1.

def create_and_display_lower_forward_diff(x_values, y_values):
    """
    Hàm này thực hiện B1, B2, và B3:
    1. Tạo, hiển thị Bảng Sai phân Tiến dạng tam giác dưới.
    2. Tô màu các giá trị trên đường chéo chính (Δ^k y_0).
    3. Tính toán và in ra vector hệ số (Δ^k y_0 / k!).
    """
    
    n = len(y_values)
    
    # --- 1. Tính toán Bảng Sai phân Tiến (Dạng tam giác dưới) ---
    # Bảng này được thiết kế để đường chéo chính là Δ^k y_0
    
    # Khởi tạo ma trận (n x n) với giá trị NaN
    table = np.full((n, n), np.nan)
    
    # Cột đầu tiên (j=0) là các giá trị y
    table[:, 0] = y_values
    
    # Tính toán các giá trị sai phân
    for j in range(1, n): # Cột (bậc sai phân)
        for i in range(j, n): # Hàng (bắt đầu từ đường chéo)
            # Công thức sai phân tiến: Δ^j y_i = Δ^{j-1} y_{i+1} - Δ^{j-1} y_i
            # Ánh xạ sang bảng tam giác dưới:
            # table[i, j] = table[i, j-1] - table[i-1, j-1]
            table[i, j] = table[i, j-1] - table[i-1, j-1]
            
    # --- B1: Tạo DataFrame Hiển thị ---
    headers = [f'y'] + [f'Δ^{i}y' for i in range(1, n)]
    df_display = pd.DataFrame(table, columns=headers)
    df_display.insert(0, 'x', x_values) # Thêm cột x vào đầu

    # Hàm tô màu đường chéo
    def highlight_diagonal(data):
        style = 'background-color: yellow; font-weight: bold; color: black;'
        style_df = pd.DataFrame('', index=data.index, columns=data.columns)
        
        # Lặp và tô màu đường chéo
        # Bỏ qua cột 'x' (cột 0), bắt đầu từ cột 'y' (cột 1)
        for i in range(len(data.index)):
            col_idx_in_df = i + 1 # Cột 'y' là 1, 'Δ^1y' là 2, ...
            if col_idx_in_df < len(data.columns):
                val = data.iloc[i, col_idx_in_df]
                if not pd.isna(val): # Chỉ tô màu ô có giá trị
                    style_df.iloc[i, col_idx_in_df] = style
        return style_df

    # Áp dụng style và định dạng
    print("--- B1: Bảng Sai phân Tiến (Dạng tam giác dưới, tô màu đường chéo) ---")
    numeric_cols = df_display.select_dtypes(include=np.number).columns
    styled_df = df_display.style.apply(highlight_diagonal, axis=None).format(
        "{:g}", 
        subset=numeric_cols, 
        na_rep="" # Hiển thị ô trống thay vì 'NaN'
    )
    
    display(styled_df)
    
    # --- B2: Lấy giá trị trên đường chéo và chia giai thừa ---
    
    # 1. Lấy các giá trị trên đường chéo (chính là các hệ số Δ^k y_0)
    raw_coeffs_delta = np.diag(table)
    
    # 2. Tạo vector tham số mới bằng cách chia cho k!
    parameter_vector = []
    for k in range(n):
        coeff_k = raw_coeffs_delta[k] / math.factorial(k)
        parameter_vector.append(coeff_k)
        
    # --- B3: In vector tham số ---
    print("\n--- B2 & B3: Vector tham số (Δ^k y_0 / k!) ---")
    print(np.array(parameter_vector))


# --- Thực thi hàm ---
try:
    # Chạy hàm để hiển thị bảng và in vector
    create_and_display_lower_forward_diff(x_coords, y_coords)
    
except NameError:
    print("LỖI: Các biến 'x_coords' và 'y_coords' chưa được định nghĩa. Vui lòng chạy Ô 1 trước.")

--- B1: Bảng Sai phân Tiến (Dạng tam giác dưới, tô màu đường chéo) ---


,x,y,Δ^1y,Δ^2y,Δ^3y,Δ^4y,Δ^5y
0,2.1,0.5597,,,,,
1,2.3,1.0378,0.4781,,,,
2,2.5,1.5337,0.4959,0.0178,,,
3,2.7,1.9847,0.451,-0.0449,-0.0627,,
4,2.9,2.3355,0.3508,-0.1002,-0.0553,0.0074,
5,3.1,2.5465,0.211,-0.1398,-0.0396,0.0157,0.0083



--- B2 & B3: Vector tham số (Δ^k y_0 / k!) ---
[ 5.59700000e-01  4.78100000e-01  8.90000000e-03 -1.04500000e-02
  3.08333333e-04  6.91666667e-05]


In [21]:
# Giả định các biến x_coords và y_coords đã tồn tại từ Ô 1.

def create_and_display_lower_forward_diff(x_values, y_values):
    """
    Hàm này thực hiện:
    1. Tạo, hiển thị Bảng Sai phân Tiến dạng tam giác dưới.
    2. Tô màu các giá trị trên đường chéo chính (Δ^k y_0).
    3. Tính toán và TRẢ VỀ vector hệ số (Δ^k y_0 / k!).
    """
    
    n = len(y_values)
    
    # --- 1. Tính toán Bảng Sai phân Tiến ---
    table = np.full((n, n), np.nan)
    table[:, 0] = y_values
    for j in range(1, n):
        for i in range(j, n):
            table[i, j] = table[i, j-1] - table[i-1, j-1]
            
    # --- B1: Tạo DataFrame Hiển thị ---
    headers = [f'y'] + [f'Δ^{i}y' for i in range(1, n)]
    df_display = pd.DataFrame(table, columns=headers)
    df_display.insert(0, 'x', x_values)

    # Hàm tô màu đường chéo
    def highlight_diagonal(data):
        style = 'background-color: yellow; font-weight: bold; color: black;'
        style_df = pd.DataFrame('', index=data.index, columns=data.columns)
        for i in range(len(data.index)):
            col_idx_in_df = i + 1
            if col_idx_in_df < len(data.columns):
                val = data.iloc[i, col_idx_in_df]
                if not pd.isna(val):
                    style_df.iloc[i, col_idx_in_df] = style
        return style_df

    # Áp dụng style và định dạng
    print("--- B1: Bảng Sai phân Tiến (Dạng tam giác dưới, tô màu đường chéo) ---")
    numeric_cols = df_display.select_dtypes(include=np.number).columns
    styled_df = df_display.style.apply(highlight_diagonal, axis=None).format(
        "{:g}", 
        subset=numeric_cols, 
        na_rep=""
    )
    display(styled_df)
    
    # --- B2: Lấy giá trị trên đường chéo và chia giai thừa ---
    raw_coeffs_delta = np.diag(table)
    parameter_vector = []
    for k in range(n):
        coeff_k = raw_coeffs_delta[k] / math.factorial(k)
        parameter_vector.append(coeff_k)
        
    final_parameter_vector = np.array(parameter_vector)
    
    # --- B3: In vector tham số ---
    print("\n--- B2 & B3: Vector tham số (Δ^k y_0 / k!) ---")
    print(final_parameter_vector)

    # >>> THAY ĐỔI QUAN TRỌNG: TRẢ VỀ (RETURN) VECTOR ĐỂ SỬ DỤNG SAU <<<
    return final_parameter_vector

# --- Thực thi hàm ---
try:
    # >>> THAY ĐỔI: Lưu kết quả trả về vào biến mới <<<
    newton_coeffs_factorial = create_and_display_lower_forward_diff(x_coords, y_coords)
    
except NameError:
    print("LỖI: Các biến 'x_coords' và 'y_coords' chưa được định nghĩa. Vui lòng chạy Ô 1 trước.")

--- B1: Bảng Sai phân Tiến (Dạng tam giác dưới, tô màu đường chéo) ---


,x,y,Δ^1y,Δ^2y,Δ^3y,Δ^4y,Δ^5y
0,2.1,0.5597,,,,,
1,2.3,1.0378,0.4781,,,,
2,2.5,1.5337,0.4959,0.0178,,,
3,2.7,1.9847,0.451,-0.0449,-0.0627,,
4,2.9,2.3355,0.3508,-0.1002,-0.0553,0.0074,
5,3.1,2.5465,0.211,-0.1398,-0.0396,0.0157,0.0083



--- B2 & B3: Vector tham số (Δ^k y_0 / k!) ---
[ 5.59700000e-01  4.78100000e-01  8.90000000e-03 -1.04500000e-02
  3.08333333e-04  6.91666667e-05]


In [22]:
# Giả định biến 'x_coords' đã được định nghĩa ở ô trên
# (Chúng ta chỉ cần 'x_coords' để biết tổng số mốc nội suy n)

try:
    # --- B1 & B2: Thiết lập thông số ma trận ---
    n = len(x_coords)
    
    # B1: Cột "c" chứa các giá trị từ 0 đến n-2 (tổng cộng n-1 giá trị)
    c_values = np.arange(0, n - 1)
    num_rows_to_calc = n - 1 # Sẽ có n-1 hàng được tính toán

    # B2: Bảng ma trận có n cột (Col_0 đến Col_n-1)
    num_cols = n 

    # --- B3: Bước khởi tạo và tính toán ---
    product_table_rows = []
    
    # Hàng đầu tiên [0, 0, ..., 1] có n phần tử
    initial_row = np.zeros(num_cols)
    initial_row[-1] = 1.0
    product_table_rows.append(initial_row)

    print("--- Bắt đầu xây dựng Bảng Tích (theo c = 0, 1, 2,...) ---")
    print(f"Số mốc nội suy n = {n}")
    print(f"Số cột trong bảng tính = {num_cols}")
    print(f"Các giá trị c (từ 0 đến n-2): {c_values.tolist()}")
    
    # --- B2 (tiếp): Lặp và tính toán các hàng ---
    # a: là hàng trước đó
    # b: là hàng đang tính
    for i in range(num_rows_to_calc):
        c = c_values[i]
        a = product_table_rows[-1] # Lấy hàng cuối cùng vừa được thêm
        b = np.zeros(num_cols)     # Hàng mới

        # Áp dụng công thức: b_k = a_{k+1} - a_k * c
        for k in range(num_cols):
            # Lấy a[k+1], nếu k là chỉ số cuối cùng thì coi như a[k+1] = 0
            a_k_plus_1 = a[k+1] if k < num_cols - 1 else 0.0 
            a_k = a[k]
            b[k] = a_k_plus_1 - (a_k * c)
            
        product_table_rows.append(b)

    # --- Hiển thị kết quả ---
    # Gán ma trận kết quả vào biến product_table_data để dùng về sau
    product_table_data = np.array(product_table_rows)
    
    # Tạo DataFrame để hiển thị cho đẹp
    df_product = pd.DataFrame(product_table_data)
    
    # Chèn cột giá trị 'c' vào đầu
    c_column_display = [np.nan] + c_values.tolist()
    df_product.insert(0, 'Giá trị c (k)', c_column_display)

    print("\n--- Bảng Tích Hoàn Chỉnh ---")
    display(df_product.style.format("{:g}", na_rep=""))

except NameError:
    print("LỖI: Biến 'x_coords' chưa được định nghĩa. Vui lòng chạy ô nạp dữ liệu trước.")

--- Bắt đầu xây dựng Bảng Tích (theo c = 0, 1, 2,...) ---
Số mốc nội suy n = 6
Số cột trong bảng tính = 6
Các giá trị c (từ 0 đến n-2): [0, 1, 2, 3, 4]

--- Bảng Tích Hoàn Chỉnh ---


,Giá trị c (k),0,1,2,3,4,5
0,,0,0,0,0,0,1
1,0,0,0,0,0,1,0
2,1,0,0,0,1,-1,0
3,2,0,0,1,-3,2,0
4,3,0,1,-6,11,-6,0
5,4,1,-10,35,-50,24,0


In [23]:
# Giả định các biến 'newton_coeffs_factorial' và 'product_table_data' đã tồn tại.

# (Hàm phụ trợ để in đa thức - Đảm bảo đã chạy ở ô trước)
try:
    _ = format_polynomial_descending
except NameError:
    print("Đang định nghĩa hàm 'format_polynomial_descending'...")
    def format_polynomial_descending(coeffs_asc):
        # ... (code hàm format) ...
        coeffs_desc = coeffs_asc[::-1]; poly_str = ""; n = len(coeffs_desc) - 1
        for i, coeff in enumerate(coeffs_desc):
            if np.isclose(coeff, 0): continue
            power = n - i; sign = " - " if coeff < 0 else " + "; coeff_abs = abs(coeff)
            if poly_str == "": sign = "" if coeff > 0 else "-"
            if np.isclose(coeff_abs, 1) and power != 0: coeff_str = ""
            else: coeff_str = f"{coeff_abs:g}"
            if power == 1: var_str = "x"
            elif power == 0: var_str = ""
            else: var_str = f"x^{power}"
            separator = "*" if coeff_str != "" and var_str != "" else ""
            poly_str += f" {sign} {coeff_str}{separator}{var_str}"
        return poly_str.strip() if poly_str else "0"

# --- B1: In Vector tham số và Bảng Tích ---
print("--- B1: Vector tham số (đã chia giai thừa) ---")
try:
    with np.printoptions(threshold=np.inf, precision=15, suppress=True):
        print(newton_coeffs_factorial)
except NameError:
    print("LỖI: Biến 'newton_coeffs_factorial' chưa được định nghĩa.")
    raise 
print("\n--- B1: Bảng Tích (Ma trận nhân) ---")
try:
    with np.printoptions(threshold=np.inf, precision=15, suppress=True, linewidth=np.inf):
        print(product_table_data)
except NameError:
    print("LỖI: Biến 'product_table_data' chưa được định nghĩa.")
    raise 
print("-" * 50)

# --- B2: Thực hiện phép nhân ma trận ---
print("--- B2: Thực hiện phép nhân ---")
print("Vector_Tham_Số @ Bảng_Tích")

try:
    # >>> THAY ĐỔI Ở ĐÂY: Đổi tên biến cho đúng <<<
    # Phép nhân này cho ra hệ số bậc GIẢM DẦN (An, A(n-1), ..., A0)
    final_coeffs_descending = newton_coeffs_factorial @ product_table_data
    
    # --- B3: In kết quả và đa thức ---
    print("\n--- Kết quả Vector hệ số cuối cùng (Hệ số P(t) bậc GIẢM DẦN) ---")
    with np.printoptions(threshold=np.inf, precision=15, suppress=True):
        print(final_coeffs_descending)
        
    print("\n--- B3: Đa thức nội suy P(t) hoàn chỉnh ---")
    # Hàm 'format_polynomial_descending' cần hệ số bậc TĂNG DẦN (A0, ..., An)
    # Vì vậy, chúng ta đảo ngược (final_coeffs_descending[::-1]) trước khi gọi hàm
    print(f"P(t) =", format_polynomial_descending(final_coeffs_descending[::-1]).replace('x', 't'))

except ValueError as e:
    print(f"\nLỖI KHI NHÂN MA TRẬN: {e}")
    print(f"Kích thước Vector tham số: {newton_coeffs_factorial.shape}")
    print(f"Kích thước Bảng Tích: {product_table_data.shape}")
except NameError as e:
     print(f"LỖI: Một trong các biến ('newton_coeffs_factorial' hoặc 'product_table_data') chưa được định nghĩa.")

print("=" * 50)
print("--- KẾT THÚC ---")

--- B1: Vector tham số (đã chia giai thừa) ---
[ 0.5597             0.4781             0.0089
 -0.01045            0.000308333333333  0.000069166666667]

--- B1: Bảng Tích (Ma trận nhân) ---
[[  0.   0.   0.   0.   0.   1.]
 [  0.   0.   0.   0.   1.   0.]
 [  0.   0.   0.   1.  -1.   0.]
 [  0.   0.   1.  -3.   2.   0.]
 [  0.   1.  -6.  11.  -6.   0.]
 [  1. -10.  35. -50.  24.   0.]]
--------------------------------------------------
--- B2: Thực hiện phép nhân ---
Vector_Tham_Số @ Bảng_Tích

--- Kết quả Vector hệ số cuối cùng (Hệ số P(t) bậc GIẢM DẦN) ---
[ 0.000069166666667 -0.000383333333333 -0.009879166666667
  0.040183333333334  0.44811            0.5597           ]

--- B3: Đa thức nội suy P(t) hoàn chỉnh ---
P(t) = 6.91667e-05*t^5  -  0.000383333*t^4  -  0.00987917*t^3  +  0.0401833*t^2  +  0.44811*t  +  0.5597
--- KẾT THÚC ---


## Thuật toán tính giá trị đạo hàm của P(t') tại t = t' khi mình nhập giá trị x'

In [24]:
# Giả định các biến sau đã tồn tại từ các ô trước:
# final_coeffs_ascending: Vector hệ số P(t) bậc tăng dần (từ phép nhân ma trận)
# x_coords: Mảng NumPy chứa các mốc x (đã sắp xếp tăng dần)
# h_step: Bước nhảy (đã tính ở Ô 1)

# --- 1. Nhập x' và chuyển đổi sang t' ---
try:
    x_prime_input = float(input("Nhập giá trị x' cần tính đạo hàm: "))
    
    # Lấy x0 và h từ các bước trước
    x0 = x_coords[0]
    h = h_step
    
    # Tính giá trị t' tương ứng
    t_prime = (x_prime_input - x0) / h
    
    print("-" * 50)
    print(f"Giá trị x' = {x_prime_input}")
    print(f"Mốc x0 = {x0}, Bước nhảy h = {h:g}")
    print(f"Giá trị t' tương ứng = ({x_prime_input} - {x0}) / {h:g} = {t_prime:g}")
    
except NameError:
    print("LỖI: Biến 'x_coords' hoặc 'h_step' chưa được định nghĩa.")
    print("Vui lòng chạy lại Ô 1 (Nạp dữ liệu) trước.")
    raise
except Exception as e:
    print(f"Lỗi: {e}")
    raise

# --- 2. Hàm thuật toán Horner lặp (để tính d^k P / dt^k) ---
def tinh_dao_ham_theo_t(coeffs_asc, t_val):
    """
    Tính giá trị của P(t) và các đạo hàm d^k P / dt^k
    tại một điểm t_val, sử dụng hệ số bậc TĂNG DẦN.
    """
    
    # Horner lặp hoạt động với hệ số bậc giảm dần
    coeffs_desc = coeffs_asc[::-1]
    
    print("\n" + "="*50)
    print(f"Đa thức P(t) có hệ số (bậc giảm dần):")
    with np.printoptions(threshold=np.inf, precision=15, suppress=True):
        print(coeffs_desc)
    print(f"Tính các đạo hàm tại t = {t_val:g}")
    
    current_coeffs = list(coeffs_desc)
    n = len(current_coeffs) - 1
    summary_data_t = [] # Nơi lưu kết quả đạo hàm theo t
    k_factorial = 1.0

    # Lặp qua từng cấp đạo hàm, từ k = 0 đến n
    for k in range(n + 1):
        print("\n" + "="*50 + f"\n--- BƯỚC {k+1}: TÍNH TOÁN CHO ĐẠO HÀM CẤP {k} (theo t) ---")
        
        m = len(current_coeffs)
        deg = m - 1
        
        # --- Hiển thị bảng Horner cho bước hiện tại ---
        header = [f'a_{deg-i}' for i in range(deg + 1)]
        row_coeffs = [round(c, 10) for c in current_coeffs]
        row_products = ['']
        row_results = []

        b = current_coeffs[0]
        row_results.append(round(b, 10))
        
        quotient_coeffs = [b] # Hệ số của đa thức thương Q_{k+1}(t)
        
        for i in range(1, m):
            product = b * t_val
            row_products.append(round(product, 10))
            b = product + current_coeffs[i]
            row_results.append(round(b, 10))
            if i < m - 1:
                quotient_coeffs.append(b)
        
        remainder = b # Số dư (R_k+1)
        
        df = pd.DataFrame(
            [row_coeffs, row_products, row_results],
            index=['Hệ số vào', f'Nhân với t={t_val:g}', 'Kết quả'],
            columns=header
        )
        display(df)
        
        # --- Tính giá trị đạo hàm d^k P / dt^k = k! * R_{k+1} ---
        if k > 0:
            k_factorial *= k
        
        derivative_value_t = remainder * k_factorial
        summary_data_t.append([f"d^{k}P/dt^{k}", derivative_value_t])
        
        print(f"-> Số dư R_{k+1} = {remainder:g}")
        print(f"-> Giá trị đạo hàm d^{k}P/dt^{k} = R_{k+1} * {k}! = {remainder:g} * {k_factorial} = {derivative_value_t:g}")

        # Cập nhật hệ số cho vòng lặp tiếp theo
        current_coeffs = quotient_coeffs

    # --- In bảng tổng kết cuối cùng cho P(t) ---
    column_name_value = f"Giá trị tại t = {t_val:g}"
    summary_df_t = pd.DataFrame(summary_data_t, columns=["Đạo hàm (theo t)", column_name_value])
    print("\n" + "="*50 + f"\n--- BẢNG TỔNG HỢP KẾT QUẢ (theo t) ---")
    display(summary_df_t.style.format("{:g}", subset=[column_name_value]))
    
    return summary_df_t

# --- 3. Thực thi thuật toán và chuyển đổi sang đạo hàm P(x) ---
try:
    # Tính toán các đạo hàm theo t
    summary_df_t = tinh_dao_ham_theo_t(final_coeffs_descending[::-1], t_prime)
    
    # --- 4. Chuyển đổi đạo hàm từ P(t) sang P(x) ---
    # Sử dụng quy tắc chuỗi (chain rule):
    # dP/dx = (dP/dt) * (dt/dx)
    # Vì t = (x - x0)/h => dt/dx = 1/h
    # => d^k P / dx^k = (1 / h^k) * (d^k P / dt^k)
    print("\n" + "="*50)
    print("--- CHUYỂN ĐỔI SANG ĐẠO HÀM THEO x ---")
    print(f"Sử dụng công thức: d^k P / dx^k = (1 / h^k) * (d^k P / dt^k)")
    
    derivatives_wrt_x = []
    for k in range(len(summary_df_t)):
         # Lấy giá trị đạo hàm theo t từ bảng
         dkP_dtk = summary_df_t.iloc[k, 1] 
         
         # Tính đạo hàm theo x
         dkP_dxk = (1 / (h**k)) * dkP_dtk
         derivatives_wrt_x.append([f"P^({k})(x')", dkP_dxk])

    # In bảng tổng kết cuối cùng cho P(x)
    column_name_x = f"Giá trị tại x = {x_prime_input}"
    summary_df_x = pd.DataFrame(derivatives_wrt_x, columns=["Đạo hàm (theo x)", column_name_x])
    print("\n" + "="*50 + f"\n--- BẢNG TỔNG HỢP KẾT QUẢ CHO x = {x_prime_input} (theo x) ---")
    display(summary_df_x.style.format("{:g}", subset=[column_name_x]))

except NameError:
    print("LỖI: Biến 'final_coeffs_ascending' chưa được định nghĩa.")
    print("Vui lòng chạy lại ô code thực hiện phép nhân ma trận trước.")
except Exception as e:
    print(f"Đã xảy ra lỗi: {e}")

--------------------------------------------------
Giá trị x' = 2.4
Mốc x0 = 2.1, Bước nhảy h = 0.2
Giá trị t' tương ứng = (2.4 - 2.1) / 0.2 = 1.5

Đa thức P(t) có hệ số (bậc giảm dần):
[ 0.000069166666667 -0.000383333333333 -0.009879166666667
  0.040183333333334  0.44811            0.5597           ]
Tính các đạo hàm tại t = 1.5

--- BƯỚC 1: TÍNH TOÁN CHO ĐẠO HÀM CẤP 0 (theo t) ---


,a_5,a_4,a_3,a_2,a_1,a_0
Hệ số vào,0.000069,-0.000383,-0.009879,0.040183,0.448110,0.55970
Nhân với t=1.5,,0.000104,-0.000419,-0.015448,0.037103,0.72782
Kết quả,0.000069,-0.000280,-0.010299,0.024736,0.485213,1.28752


-> Số dư R_1 = 1.28752
-> Giá trị đạo hàm d^0P/dt^0 = R_1 * 0! = 1.28752 * 1.0 = 1.28752

--- BƯỚC 2: TÍNH TOÁN CHO ĐẠO HÀM CẤP 1 (theo t) ---


,a_4,a_3,a_2,a_1,a_0
Hệ số vào,0.000069,-0.000280,-0.010299,0.024736,0.485213
Nhân với t=1.5,,0.000104,-0.000264,-0.015843,0.013338
Kết quả,0.000069,-0.000176,-0.010562,0.008892,0.498551


-> Số dư R_2 = 0.498551
-> Giá trị đạo hàm d^1P/dt^1 = R_2 * 1! = 0.498551 * 1.0 = 0.498551

--- BƯỚC 3: TÍNH TOÁN CHO ĐẠO HÀM CẤP 2 (theo t) ---


...

,a_0
Hệ số vào,0.000069
Nhân với t=1.5,
Kết quả,0.000069


-> Số dư R_6 = 6.91667e-05
-> Giá trị đạo hàm d^5P/dt^5 = R_6 * 5! = 6.91667e-05 * 120.0 = 0.0083

--- BẢNG TỔNG HỢP KẾT QUẢ (theo t) ---


,Đạo hàm (theo t),Giá trị tại t = 1.5
0,d^0P/dt^0,1.28752
1,d^1P/dt^1,0.498551
2,d^2P/dt^2,-0.0142271
3,d^3P/dt^3,-0.0637375
4,d^4P/dt^4,0.00325
5,d^5P/dt^5,0.0083



--- CHUYỂN ĐỔI SANG ĐẠO HÀM THEO x ---
Sử dụng công thức: d^k P / dx^k = (1 / h^k) * (d^k P / dt^k)

--- BẢNG TỔNG HỢP KẾT QUẢ CHO x = 2.4 (theo x) ---


,Đạo hàm (theo x),Giá trị tại x = 2.4
0,P^(0)(x'),1.28752
1,P^(1)(x'),2.49276
2,P^(2)(x'),-0.355677
3,P^(3)(x'),-7.96719
4,P^(4)(x'),2.03125
5,P^(5)(x'),25.9375
